## Implement Ensemble Agent

Ensemble agent that controlls all the sub price agents of different models

In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import logging
import random
from dotenv import load_dotenv
from huggingface_hub import login
import matplotlib.pyplot as plt
import numpy as np
import pickle
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
import modal

### Internal Classes
from testing import Tester
from agents.ensemble_agent import EnsembleAgent

/Volumes/VTG/Dev/C_5/Projects/week8/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
with open("test.pkl", "rb") as f: 
    test = pickle.load(f)

In [5]:
DB = "products_vectorstore/"

client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection("product")

In [6]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [7]:
logging.basicConfig(level=logging.INFO)

In [8]:
ensemble_agent = EnsembleAgent(collection)

INFO:root:[Ensemble Agent] Initializing Ensemble Agent...
INFO:root:[Specialist Agent] Specialist Agent is initializing -- connecting to Modal...
INFO:root:[Specialist Agent] Specialist Agent is ready!
INFO:root:[Frontier Agent] Initializing Frontier Agent...
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready!
INFO:root:[Neural Network Agent] Neural Network Agent is initializing...
INFO:root:[Neural Network Agent] Neural Network Agent is ready!
INFO:root:[Ensemble Agent] Ensemble Agent is ready!


In [9]:
ensemble_agent.price("IGK Good Behavior Frizz & Flyaway Hair Taming Wand – Lightweight Gel Control Stick for Smooth, Sleek Hair, Anti-Frizz Styling, 0.24 oz")

INFO:root:[Ensemble Agent] Running Ensemble Agent - collaborating with specialist, frontier and neural network agents...
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model... If this is a cold start, this might take some time to wake it up
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $7.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.22it/s]
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5-mini with context including 5 similar products...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $22.00
INFO:root:[Neural Network Agent] Neural Network Agent is p

17.5

In [10]:
test[251].price

79.8

In [11]:
len(test)

19000

In [12]:
Tester.test(ensemble_agent.price, test[:250])

INFO:root:[Ensemble Agent] Running Ensemble Agent - collaborating with specialist, frontier and neural network agents...
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model... If this is a cold start, this might take some time to wake it up
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $220.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.67it/s]
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5-mini with context including 5 similar products...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $174.95
INFO:root:[Neural Network Agent] Neural Network Agent i

1: Guess: $174.30 Truth: $249.99 Error: $75.69 SLE: 0.13 Item: Medicom Batman Hush Black Suit Version B...


INFO:root:[Specialist Agent] Specialist Agent completed - predicting $7.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.20it/s]
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5-mini with context including 5 similar products...


KeyboardInterrupt: 